In [1]:
import re
import math
import pandas as pd
import utils

In [3]:
import utils

# Manually specified Lennard-Jones parameters for Ni-Ni, O-O, and Ni-O interactions
manual_lj_dict = {
    ('Ni', 'Ni'): {'epsilon': 0.1, 'sigma': 3.0},
    ('Ni', 'O'): {'epsilon': 2.032, 'sigma': 2.32},  
    ('O', 'O'): {'epsilon': 0.210, 'sigma': 3.05}
}

buckingham_params = {
    ('Ni', 'Ni'): {
        'A': 0.0,  # A in kcal/mol
        'rho': 1.0,  # rho in angstroms
        'C': 0.0  # C in kcal/mol * angstrom^6
    },
    ('Ni', 'O'): {
        'A': 754.92,  # A in kcal/mol
        'rho': 0.3277,  # rho in angstroms
        'C': 0.00  # C in kcal/mol * angstrom^6
    },
    ('O', 'O'): {
        'A': 22764.3,  # A in kcal/mol
        'rho': 0.149,  # rho in angstroms
        'C': 27.89  # C in kcal/mol * angstrom^6
    }
}


# Step 1: Read OPLS-AA parameters
opls_data = utils.read_opls_aa_parameters('./mnt/data/ligand_oplsaa.lmp')

# Extract ligand atom types
ligand_atoms = [entry['atom_type'] for entry in opls_data['pair']]

# Step 2: Format OPLS-AA parameters
lj_params_str = utils.format_lj_parameters(opls_data)
bond_params_str = utils.format_bond_parameters(opls_data)
angle_params_str = utils.format_angle_parameters(opls_data)
dihedral_params_str = utils.format_dihedral_parameters(opls_data)
improper_params_str = utils.format_improper_parameters(opls_data)
#buckingham_lines = utils.format_buckingham_parameters(buckingham_params)

# Step 2 debug Check if opls_data and manual_lj_dict are defined correctly
#print("OPLS Data:", opls_data)
#print("Manual LJ Dictionary:", manual_lj_dict)

# Step 3: Apply combining rules
combined_lj_lines = utils.apply_combining_rules(opls_data, manual_lj_dict, ligand_atoms, mixing_method='geom')

# Step 4: Generate Ni-O parameters
ni_o_lines = [
    f"pair_coeff Ni Ni lj/cut/coul/long {manual_lj_dict[('Ni', 'Ni')]['epsilon']:.3f} {manual_lj_dict[('Ni', 'Ni')]['sigma']:.3f} # Ni-Ni",
    f"pair_coeff O O lj/cut/coul/long {manual_lj_dict[('O', 'O')]['epsilon']:.3f} {manual_lj_dict[('O', 'O')]['sigma']:.3f} # O-O",
    f"pair_coeff Ni O lj/cut/coul/long {manual_lj_dict[('Ni', 'O')]['epsilon']:.3f} {manual_lj_dict[('Ni', 'O')]['sigma']:.3f} # Ni-O"
]

# Step 5: Combine all parameters
combined_data = utils.combine_parameters(
    lj_params_str,
    bond_params_str,
    angle_params_str,
    dihedral_params_str,
    improper_params_str,
    combined_lj_lines,
    ni_o_lines,
    mixing_method='geom'
)

# Step 6: Save the combined parameters
utils.save_combined_parameters('./mnt/data/force_field_settings_lammps.lmp', combined_data)
print(f"Combined parameters saved to force_field_settings_lammps.lmp")

Finished parsing OPLS-AA parameters.
Debug: Using mixing method: geom
Combined parameters saved to force_field_settings_lammps.lmp


In [2]:
import os
import utils

# Manually specified Lennard-Jones parameters for Ni-Ni, O-O, and Ni-O interactions
manual_lj_dict = {
    ('Ni', 'Ni'): {'epsilon': 0.1, 'sigma': 3.0},
    ('Ni', 'O'): {'epsilon': 2.032, 'sigma': 2.32},  
    ('O', 'O'): {'epsilon': 0.210, 'sigma': 3.05}
}

# Function to reorder parameters
def reorder_parameters(parameters, max_atom_type, param_type):
    reordered = []
    for param in parameters:
        if param_type == 'pair':
            reordered.append({
                'atom_type': param['atom_type'] + max_atom_type,
                'epsilon': param['epsilon'],
                'sigma': param['sigma']
            })
        else:
            reordered.append(tuple([param[0] + max_atom_type] + list(param[1:])))
    return reordered

# Function to merge OPLS data
def merge_opls_data(ligand1_data, ligand2_data):
    max_atom_type_pair = max([item['atom_type'] for item in ligand1_data['pair']])
    max_atom_type_bond = max([item[0] for item in ligand1_data['bond']])
    max_atom_type_angle = max([item[0] for item in ligand1_data['angle']])
    max_atom_type_dihedral = max([item[0] for item in ligand1_data['dihedral']])
    max_atom_type_improper = max([item[0] for item in ligand1_data['improper']])

    combined_data = {
        'pair': ligand1_data['pair'] + reorder_parameters(ligand2_data['pair'], max_atom_type_pair, 'pair'),
        'bond': ligand1_data['bond'] + reorder_parameters(ligand2_data['bond'], max_atom_type_bond, 'bond'),
        'angle': ligand1_data['angle'] + reorder_parameters(ligand2_data['angle'], max_atom_type_angle, 'angle'),
        'dihedral': ligand1_data['dihedral'] + reorder_parameters(ligand2_data['dihedral'], max_atom_type_dihedral, 'dihedral'),
        'improper': ligand1_data['improper'] + reorder_parameters(ligand2_data['improper'], max_atom_type_improper, 'improper')
    }
    return combined_data

# Read OPLS-AA parameters for two ligands
ligand1_opls_data = utils.read_opls_aa_parameters('./mnt/data/ligand_oplsaa.lmp')
ligand2_opls_data = utils.read_opls_aa_parameters('./mnt/data/ligand2_oplsaa.lmp')

# Reorder and merge ligand parameters
combined_opls_data = merge_opls_data(ligand1_opls_data, ligand2_opls_data)

# Format OPLS-AA parameters
lj_params_str = utils.format_lj_parameters(combined_opls_data)
bond_params_str = utils.format_bond_parameters(combined_opls_data)
angle_params_str = utils.format_angle_parameters(combined_opls_data)
dihedral_params_str = utils.format_dihedral_parameters(combined_opls_data)
improper_params_str = utils.format_improper_parameters(combined_opls_data)

# Get list of unique ligand atom types
ligand_atoms = list(set(item['atom_type'] for item in combined_opls_data['pair']))

# Apply combining rules
combined_lj_lines = utils.apply_combining_rules(combined_opls_data, manual_lj_dict, ligand_atoms, mixing_method='geom')

# Generate Ni-O parameters
ni_o_lines = [
    f"pair_coeff Ni Ni lj/cut/coul/long {manual_lj_dict[('Ni', 'Ni')]['epsilon']:.3f} {manual_lj_dict[('Ni', 'Ni')]['sigma']:.3f} # Ni-Ni",
    f"pair_coeff O O lj/cut/coul/long {manual_lj_dict[('O', 'O')]['epsilon']:.3f} {manual_lj_dict[('O', 'O')]['sigma']:.3f} # O-O",
    f"pair_coeff Ni O lj/cut/coul/long {manual_lj_dict[('Ni', 'O')]['epsilon']:.3f} {manual_lj_dict[('Ni', 'O')]['sigma']:.3f} # Ni-O"
]

# Combine all parameters
combined_data = utils.combine_parameters(
    lj_params_str,
    bond_params_str,
    angle_params_str,
    dihedral_params_str,
    improper_params_str,
    combined_lj_lines,
    ni_o_lines,
    mixing_method='geom'
)

# Save the combined parameters
output_file = './mnt/data/force_field_settings_lammps_double.lmp'
utils.save_combined_parameters(output_file, combined_data)
print(f"Combined parameters saved to {output_file}")

Finished parsing OPLS-AA parameters.
Finished parsing OPLS-AA parameters.
Debug: Using mixing method: geom
Combined parameters saved to ./mnt/data/force_field_settings_lammps_double.lmp


In [ ]:
# Define the force field settings section
force_field_settings = """
# Include the force field settings here
bond_style      harmonic
angle_style     harmonic
dihedral_style  opls  
improper_style  harmonic
pair_style      lj/cut/coul/long 10.0 8.0 
kspace_style    pppm 1e-4
kspace_modify   slab 3.0
"""

# Path to the combined_ligand_nio.lmp file
file_path_combined = "./mnt/data/force_field_settings_lammps.lmp"

# Read the contents of the provided combined_ligand_nio.lmp file
with open(file_path_combined, 'r') as file:
    combined_ligand_nio_content = file.read()

# Attach the force field settings to the top of the file content
combined_ligand_nio_with_header = force_field_settings + combined_ligand_nio_content

# Save the modified content to a new file
output_file_path = "./mnt/data/force_field_settings_lammps_with_header.lmp"
with open(output_file_path, 'w') as file:
    file.write(combined_ligand_nio_with_header)

print(f"Modified file saved as {output_file_path}")

In [ ]:
#Lets check how reasonable our parameters are
# Read the LAMMPS file
with open('./mnt/data/force_field_settings_lammps.in', 'r') as file:
    lines = file.readlines()

    #Obsolete, we quit using buckingham directly
# Parse and verify parameters
#lj_params = utils.parse_lj_parameters(lines)
#buckingham_params = utils.parse_buckingham_parameters(lines)
#utils.verify_opls_aa_params(lj_params)
#utils.verify_buckingham_params(buckingham_params)
#utils.verify_combined_params(lj_params, lines)

In [ ]:
#Use GPT 4 to run the checks after this

In [ ]:
#Keeping this in reserve, possibly better

def read_opls_aa_parameters(filepath):
    """
    Read OPLS-AA parameters from a file.
    
    Args:
        filepath (str): Path to the OPLS-AA parameter file.
        
    Returns:
        dict: Dictionary of OPLS-AA parameters.
    """
    opls_data = {
        'pair': {},
        'bond': {},
        'angle': {},
        'dihedral': {},
        'improper': [],
    }
    
    with open(filepath, 'r') as file:
        lines = file.readlines()
    
    section = None
    improper_start = False
    
    for line in lines:
        line = line.split('#')[0].strip()  # Remove comments and extra whitespace
        if not line:
            continue
        
        if line.startswith('Pair Coeffs'):
            section = 'pair'
            continue
        elif line.startswith('Bond Coeffs'):
            section = 'bond'
            continue
        elif line.startswith('Angle Coeffs'):
            section = 'angle'
            continue
        elif line.startswith('Dihedral Coeffs'):
            section = 'dihedral'
            continue
        elif line.startswith('Improper Coeffs'):
            section = 'improper'
            improper_start = True
            continue
        
        if section == 'pair' and section is not None:
            parts = line.split()
            if len(parts) >= 3:
                try:
                    atom_type = int(parts[0])
                    epsilon = float(parts[1])
                    sigma = float(parts[2])    
                    opls_data[section][atom_type] = {'epsilon': epsilon, 'sigma': sigma}
                except ValueError as e:
                    print(f"Error parsing line: {line.strip()}, error: {e}")
        
        elif section == 'bond' and section is not None:
            parts = line.split()
            if len(parts) >= 3:
                try:
                    bond_type = int(parts[0])
                    k = float(parts[1])
                    r0 = float(parts[2])  # nm
                    opls_data[section][bond_type] = (k, r0)
                except ValueError as e:
                    print(f"Error parsing line: {line.strip()}, error: {e}")
        
        elif section == 'angle' and section is not None:
            parts = line.split()
            if len(parts) >= 3:
                try:
                    angle_type = int(parts[0])
                    k = float(parts[1])
                    theta0 = float(parts[2])  # degrees
                    opls_data[section][angle_type] = (k, theta0)
                except ValueError as e:
                    print(f"Error parsing line: {line.strip()}, error: {e}")
        
        elif section == 'dihedral' and section is not None:
            parts = line.split()
            if len(parts) >= 5:
                try:
                    dihedral_type = int(parts[0])
                    v1 = float(parts[1])
                    v2 = float(parts[2])
                    v3 = float(parts[3])
                    v4 = float(parts[4])
                    opls_data[section][dihedral_type] = (v1, v2, v3, v4)
                except ValueError as e:
                    print(f"Error parsing line: {line.strip()}, error: {e}")
        
        elif section == 'improper' and improper_start:
            parts = line.split()
            if len(parts) == 4:
                try:
                    improper_type = int(parts[0])
                    k = float(parts[1])
                    sign = int(parts[2])
                    multiplicity = int(parts[3])
                    opls_data[section].append((improper_type, k, sign, multiplicity))
                except ValueError as e:
                    print(f"Error parsing line: {line.strip()}, error: {e}")
            else:
                improper_start = False
    
    print("Finished parsing OPLS-AA parameters.")
    return opls_data


In [36]:
import os
import utils

# Manually specified Lennard-Jones parameters for Ni-Ni, O-O, and Ni-O interactions
manual_lj_dict = {
    ('Ni', 'Ni'): {'epsilon': 0.1, 'sigma': 3.0},
    ('Ni', 'O'): {'epsilon': 2.032, 'sigma': 2.32},  
    ('O', 'O'): {'epsilon': 0.210, 'sigma': 3.05}
}

# Read OPLS-AA parameters for two ligands
ligand1_opls_data = utils.read_opls_aa_parameters('./mnt/data/ligand_oplsaa.lmp')
ligand2_opls_data = utils.read_opls_aa_parameters('./mnt/data/ligand2_oplsaa.lmp')

# Combine ligand parameters using the | operator for dictionaries
combined_opls_data = ligand2_opls_data | ligand1_opls_data 

Finished parsing OPLS-AA parameters.
Finished parsing OPLS-AA parameters.
